In [9]:
# Sirve para transformar imagenes en cadenas de caracteres y viceversa con el algoritmo base64
import base64
# Realiza peticiones 
import requests
# Nos permite accede al DOM de una web usando XPATH
from lxml import html
# Comunicación con el sistema operativo
import subprocess
# Gestión de archivos CSV, codecs y "cadenas seguras" no tener problemas con las codificaciones de caracteres
import csv, codecs, cStringIO
# Ordenes al sistema operativo
import os

In [10]:
# Realizamos una peticion a una web que tiene información de interés
jardin = requests.get(
     'https://www.ferrokey.eu/jardin/sort-by/manufacturer/sort-direction/asc.html'
)
# Generamos el arbol DOM
tree = html.fromstring(jardin.content)

In [11]:
# Mediante el visualizador de código del navegador, localizamos la ruta xpath de cada uno de los datos que necesitamos.
marca = tree.xpath('//*/div[@class="item-box-footer"]/div[3]/strong/text()')
ean = tree.xpath('//*/div[@class="item-box-footer"]/div[2]/strong/text()')
nombre = tree.xpath('//*/div[@class="item-box-footer"]/h2/a/@title')
detalle = tree.xpath('//*/div[@class="item-box-footer"]/h2/a/@href')
categoria = tree.xpath('//*/div[@class="category-title"]/h1/text()')
foto = tree.xpath('//*/a[@class="product-image-inner"]/img/@src')
precio = tree.xpath('//*/span[@class="price"]/text()')
eanNombre = tree.xpath('//*/div[@class="item-box-footer"]/div[2]/strong/text() | //*/div[@class="item-box-footer"]/h2/a/@title')   

In [12]:
# Creamos una función para pasar una petición y codificar la respues en tipo string codificación base64
def urlbase64(url):
    response = requests.get(url)
    uri = base64.b64encode(response.content)
    return uri
# Establecemos una ruta en la cual guardaremos las fotos obtenidas. Si no existe, la crea.
ruta = r'/Users/ivang/scraping_ivang/jardin2/fotos'
if not os.path.exists(ruta):
    os.makedirs(ruta)

In [8]:
# Establecemos el archivo csv en el cual introducimos las cabeceras.
with open('/Users/ivang/scraping_ivang/jardin2/jardin.csv', 'w') as csvfile:
    # Establecemos las nombres de los campos de las cabeceras.
    fieldnames = ['ean', 'marca', 'nombre', 'precio', 'categoria','urlDetalle', 'urlFoto', 'fotoBase64']
    # Escribimos la linea anterior en el archivo csv.
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    #delimiter=' '
    #quotechar='|'
    writer.writeheader()
    # Iteramos el numero de productos que queramos sacar.
    for i in list(range(25)):
        # Establecemos el nombre de las imagenes a nuestro gusto, y la codificamos en base64
        with open("/Users/ivang/scraping_ivang/jardin2/fotos/imagenjardin{}codigo{}.jpg".format(i, ean[i]), "wb") as fh:
            try:
                fh.write(urlbase64(foto[i]).decode('base64'))
                # Escribimos en cada línea la información que necesitamos
                writer.writerow({'ean': ean[i], 'marca': marca[i], 'nombre':nombre[i], 'precio':precio[i].encode('utf-8'), 'categoria':'jardin', 'urlFoto':foto[i], 'urlDetalle':detalle[i], 'fotoBase64':urlbase64(foto[i])})
            except:
                pass

IndexError: list index out of range